### 主成分分析
参考文献:<br>
> 井出剛. 2015. 入門機械学習による異常検知. コロナ社. pp.123-139. <br>
> MacGregor JF, Kourti T. 1995. Statistical process control of multivariate processes. Control Eng. Practice. 3: 403-414.

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from typing import Union, Literal, Annotated

#### PCA法の特徴
1. 各変数の情報からうまく基底をつくりだすことにより、多次元の有する情報を低次元に落とし込むことが可能<br>
2. 上記より、不要な変数の寄与を取り除くことができる
3. 上記より、多次元の情報を直観的に理解しやすくすることができる

&nbsp;

ただし、以下点については注意が必要である。
1. 変数の線形和であらわされた新たな変数（主成分）は、必ずしも解釈が容易ではない
2. 当然ではあるが、線形を仮定したモデルなので、非線形的な特徴はうまく利用できないことがある
3. 各変数の寄与は一般に不明である ※1
4. 閾値設定は経験的なものとなる ※2

&nbsp;

**<注釈>** <br>
※1 各変数の寄与の算出はできない、というわけではないらしく、例えば以下には算出方法の記載が存在する <br>
> https://www.jmp.com/support/help/ja/17.2/index.shtml#page/jmp/statistical-details-for-contributions.shtml# <br>

※2 井出には記載がないものの、ベータ分布やF分布で閾値決めをしている例も存在。<br>
> https://www.jmp.com/support/help/ja/17.2/index.shtml#page/jmp/statistical-details-for-limits.shtml#

### 主成分分析による異常検知手順
#### 1. 標本平均を算出する
対象とするデータが、 $M$ 次元を有する変数 $N$ 個の標本よりなる、 $D = \{ \mathbfit{x}^{(1)}, \cdots , \mathbfit{x}^{(N)} \}$ であるとする。<br>
その時の標本平均 $\hat{\boldsymbol{\mu}}$ は以下のようになる。
$$
\hat{\boldsymbol{\mu}} = \frac{1}{N} \sum_{n=1}^N \mathbfit{x}^{(n)}
$$
また、原点調整した $x$ を行列として表現したものを $\mathbf{\tilde{X}}$ とすると、これはデータ行列 $ \mathbf{X} \equiv [\mathbfit{x}^{(1)}, \cdots , \mathbfit{x}^{(N)}] $ と中心化行列 $\mathbf{H}_N$ を用いて以下のように算出を行っておく。<br>
$$
\mathbf{\tilde{X}} \equiv \mathbf{X} \mathbf{H}_N = [\mathbfit{x}^{(1)} - \boldsymbol{\hat{\mu}}, \cdots, \mathbfit{x}^{(N)} - \boldsymbol{\hat{\mu}}] 
$$
$$
\mathbf{X} \equiv 
\begin{pmatrix} 
  x^{(1)}_1 & x^{(2)}_1 & \dots  & x^{(N)}_1 \\
  x^{(1)}_2 & x^{(2)}_2 & \dots  & x^{(N)}_2 \\
  \vdots & \vdots & \ddots & \vdots \\
  x^{(1)}_M & x^{(2)}_M & \dots  & x^{(N)}_M
\end{pmatrix} 
$$

$$
\mathbf{H}_N \equiv \mathbf{I} - \frac{1}{N} \mathbfit{1} \mathbfit{1}^\top
=
\begin{pmatrix} 
  1-\frac{1}{N} & -\frac{1}{N} & \dots  & -\frac{1}{N} \\
  -\frac{1}{N} & 1-\frac{1}{N}  & \dots  & -\frac{1}{N} \\
  \vdots & \vdots & \ddots & \vdots \\
  -\frac{1}{N} & -\frac{1}{N} & \dots  & 1-\frac{1}{N} 
\end{pmatrix} 
$$

$\mathbf{I}$ は単位行列、$\mathbfit{1}$ はすべての要素が1からなるベクトルである。

#### 2. 特異値分解を用いて、散布行列固有値、固有ベクトルを算出する
散布行列とは以下に定義される行列である。<br>
$$
\mathbf{S} \equiv \sum_{n=1}^N (\mathbfit{x^{(n)}} - \boldsymbol{\hat{\mu}})(\mathbfit{x^{(n)}} - \boldsymbol{\hat{\mu}})^\top
$$
主成分分析では、$D$ ある正規直交規定 $\mathbfit{u}_1, \cdots, \mathbfit{u}_M$ で表現できるとしたときに、 $\mathbfit{u}$ に沿った分散が最大となる、且つ $\mathbfit{u}^\top \mathbfit{u} =1$ を満たす最適化問題として定式化することができる。<br>
これは散布行列の固有値問題に帰着する。下式では、１つの固有ベクトルを算出する場合に対応した例である。<br>
$$
\mathbf{S}\mathbfit{u} = \lambda \mathbfit{u}
$$
一方で、 $\mathbf{\tilde{X}}$ に特異値分解（SVD）という手法を適用することで同様の結果をえることが可能である。<br>

$$
\mathbf{\tilde{X}} = \mathbf{U}_M \mathbf{\Lambda}_M^{1/2} \mathbf{V}_M^\top
$$
$$
\mathbf{U}_M \equiv [\mathbfit{u}_1, \cdots, \mathbfit{u}_M] =
\begin{pmatrix} 
  u_{1_1} & u_{2_1} & \dots  & u_{M_1} \\
  u_{1_2} & u_{2_2} & \dots  & u_{M_2} \\
  \vdots & \vdots & \ddots & \vdots \\
  u_{1_M} & u_{2_M} & \dots  & u_{M_M}
\end{pmatrix} 
$$
$$
\mathbf{\Lambda}_M = \mathrm{diag}(\lambda_1, \cdots, \lambda_M) =
\begin{pmatrix} 
  \lambda_1 & 0 & \dots  & 0 \\
  0 & \lambda_2 & \dots  & 0 \\
  \vdots & \vdots & \ddots & \vdots \\
  0 & 0 & \dots  & \lambda_M
\end{pmatrix} 
$$
$$
\mathbf{V}_M \equiv [\mathbfit{v}_1, \cdots, \mathbfit{v}_M] =
\begin{pmatrix} 
  v_{1_1} & v_{2_1} & \dots  & v_{M_1} \\
  v_{1_2} & v_{2_2} & \dots  & v_{M_2} \\
  \vdots & \vdots & \ddots & \vdots \\
  v_{1_N} & v_{2_N} & \dots  & v_{M_N}
\end{pmatrix} 
$$
上記では、$\mathbf{U}_M, \mathbf{\Lambda}_M$ が $M \times M$ 行列、 $\mathbf{V}_M$ が $N \times M$ 行列である。ただし上記は、サンプル数が次元数よりも大きい場合である。<br>
上記はランク落ちがない場合であり、ランク落ちでそのランクが $r$ のときは、$\mathbf{U}_r$ は $M \times r$ 行列、 $\mathbf{\Lambda}_r$ は $r \times r$ 行列、 $\mathbf{V}_r$ は $N \times r$ 行列となる。<br>
PCAにおける固有ベクトルは $\mathbf{U}_M$ 、固有値は $\mathbf{\Lambda}_M$ 、主成分スコアは $\mathbf{V}_M$ に対応する。<br>
SVDの場合、固有値は平方根の形で算出されるので、2乗する必要がある。<br>
実際には、主成分として上位m個のみを使うことが多い。mの決め方として、固有値の累積寄与率が80-90%あたりとなる数mを経験的に選択することとなる。

#### 3. 異常度の定義
一般的に用いられる異常度は2種類存在する。<br>
1つ目は、主成分空間に射影した後もとに空間に再射影した際に生じる誤差を異常値とした、再構成誤差（Q統計量）である。<br>
2つ目は、主成分空間に射影した後、その空間内でどの程度正解データから離れているかという距離を利用したT2統計量である。直観的には、主成分空間内におけるマハラノビス距離を利用した異常検知、という雰囲気である※。<br>
※ただし、PCAによる異常検知とHotelling T2法の対応関係からは、むしろ再構成誤差がT2法に対応するものとなる（井出. 2015.）。<br>

##### 3.1. 再構成誤差による異常度（Q統計量）
任意の入力を $\mathbfit{x}'$ とすると、PCAにより生じた再構成誤差は以下のように定義することができる。<br>
$$
\mathbfit{x'} - \hat{\boldsymbol{\mu}} = \mathbfit{x}'_{(1)} + \mathbfit{x}'_{(2)}
$$
上記で、 $\mathbfit{x'}_{(1)}$ はPCAにより表現できなかった成分、$\mathbfit{x'}_{(2)}$ はPCAにより表現できた成分に対応する。<br>
$\mathbfit{x}'_{(2)}$ は、一度主成分空間に射影した後、再度元の空間に射影した成分であるから、以下のように計算できる。ただし、以下の例では、主成分数をmとしている。<br>
$$
\mathbfit{x}'_{(2)} = \sum_{i=1}^m \mathbfit{u_i} \mathbfit{u_i}^\top (\mathbfit{x}' - \hat{\boldsymbol{\mu}}) = \mathbf{U}_m \mathbf{U}_m^\top (\mathbfit{x}' - \hat{\boldsymbol{\mu}}) 
$$
よって、$\mathbfit{x}'_{(1)}$ は以下のようになる。<br>
$$
\mathbfit{x}'_{(1)} = (\mathbfit{x}'- \hat{\boldsymbol{\mu}}) - \mathbfit{x}'_{(2)} = [\mathbf{1}_M - \mathbf{U}_M \mathbf{U}_M^\top](\mathbfit{x}'-\hat{\boldsymbol{\mu}})
$$
従って、異常の度合いは $\mathbfit{x}'_{(1)}$ の大きさを評価すればよく、その2乗である $\| \mathbfit{x}'_{(1)} \|^2$ を異常値として採用できる。<br>
$$ 
a(\mathbfit{x}') = \| \mathbfit{x}'_{(1)} \|^2 = \mathbfit{x}'^\top_{(1)} \mathbfit{x}'_{(1)} 
$$

$$
= (\mathbfit{x}'- \hat{\boldsymbol{\mu}})^\top [\mathbf{1}_M - \mathbf{U}_M \mathbf{U}_M^\top][\mathbf{1}_M - 
    \mathbf{U}_M \mathbf{U}_M^\top] (\mathbfit{x}'- \hat{\boldsymbol{\mu}}) 
$$

$$ 
= (\mathbfit{x}'- \hat{\boldsymbol{\mu}})^\top [\mathbf{1}_M -2\mathbf{U}_M \mathbf{U}_M^\top + 
    \mathbf{U}_M \mathbf{U}_M^\top\mathbf{U}_M \mathbf{U}_M^\top] (\mathbfit{x}'- \hat{\boldsymbol{\mu}})
$$

$$ 
= (\mathbfit{x}'- \hat{\boldsymbol{\mu}})^\top [\mathbf{1}_M -\mathbf{U}_M \mathbf{U}_M^\top] (\mathbfit{x}'- \hat{\boldsymbol{\mu}}) 
    \quad (\because \mathbf{U}_M^\top \mathbf{U}_M = \mathbf{1}_M)
$$

##### 3.2. T2統計量
任意の入力を $\mathbfit{x}'$ とすると、PCAの固有ベクトルへの射影は以下のようになる。<br>
$$
\mathbfit{z} = \sum_{i=1}^m \mathbfit{u}^\top_{i} (\mathbfit{x}'-\hat{\boldsymbol{\mu}}) = \mathbf{U}^\top_m (\mathbfit{x}'-\hat{\boldsymbol{\mu}})
$$
この $\mathbfit{z}$ に対し、Hotelling T2法を適用したものが、T2統計量にあたる異常度である。<br>
$$
a(\mathbfit{z}) = \mathbfit{z}^\top \boldsymbol{\Lambda}_m^{-1} \mathbfit{z}
$$

In [5]:
np.linalg.norm(np.ones((2, 3)))

np.float64(2.449489742783178)